In [13]:
!pip install yfinance ta xgboost plotly kaleido scikit-learn tensorflow
# Import all necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta
import ta
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
plt.style.use('default')
sns.set_palette("husl")

class CompleteBitcoinPredictor:
    def __init__(self):
        self.data = None
        self.features_df = None
        self.models = {}
        self.results = {}
        self.test_data = {}

    def collect_data(self, period='2y'):
        print("📥 Collecting Bitcoin data...")
        try:
            btc = yf.download('BTC-USD', period=period, progress=False)
            if btc.empty:
                print("❌ No data received from Yahoo Finance")
                return None

            btc.reset_index(inplace=True)
            self.data = btc
            print(f"✅ Data collected: {len(btc)} records")

            # Fixed date formatting - convert to Python datetime first
            start_date = pd.to_datetime(self.data['Date'].min()).strftime('%Y-%m-%d')
            end_date = pd.to_datetime(self.data['Date'].max()).strftime('%Y-%m-%d')
            current_price = float(self.data['Close'].iloc[-1])

            print(f"📅 Date range: {start_date} to {end_date}")
            print(f"💰 Current price: ${current_price:.2f}")
            return btc
        except Exception as e:
            print(f"❌ Error collecting data: {e}")
            return None

    def create_features(self):
        print("🔧 Creating features...")
        if self.data is None:
            print("❌ No data available. Please collect data first.")
            return None

        df = self.data.copy()
        df['Returns'] = df['Close'].pct_change()
        df['High_Low_Ratio'] = df['High'] / df['Low']
        df['Volume_Change'] = df['Volume'].pct_change()
        for window in [5, 10, 20, 50]:
            df[f'SMA_{window}'] = df['Close'].rolling(window=window).mean()
            df[f'Close_SMA_{window}_ratio'] = df['Close'] / df[f'SMA_{window}']

        try:
            df['RSI'] = ta.momentum.rsi(df['Close'], window=14)
            df['MACD'] = ta.trend.macd_diff(df['Close'])
            df['BB_upper'] = ta.volatility.bollinger_hband(df['Close'])
            df['BB_lower'] = ta.volatility.bollinger_lband(df['Close'])
            df['BB_width'] = (df['BB_upper'] - df['BB_lower']) / df['Close']
        except Exception as e:
            print(f"⚠️ Warning: Error creating some technical indicators: {e}")
        for lag in [1, 2, 3, 7]:
            df[f'Close_lag_{lag}'] = df['Close'].shift(lag)
            df[f'Returns_lag_{lag}'] = df['Returns'].shift(lag)
        df['Volatility_7d'] = df['Returns'].rolling(7).std()
        df['Volatility_30d'] = df['Returns'].rolling(30).std()
        df['Volume_MA_7'] = df['Volume'].rolling(7).mean()
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        df['Month'] = df['Date'].dt.month
        df['Quarter'] = df['Date'].dt.quarter
        df['Bull_Market'] = (df['Close'] > df['SMA_50']).astype(int)
        df['Price_Momentum_5'] = df['Close'].pct_change(5)
        df['Price_Momentum_10'] = df['Close'].pct_change(10)
        df['Volume_Price_Trend'] = df['Volume'] * df['Returns']
        df = df.dropna()

        if len(df) == 0:
            print("❌ No data remaining after feature creation")
            return None
        feature_cols = [col for col in df.columns
                       if col not in ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

        self.features_df = df[feature_cols + ['Close', 'Date']]
        print(f"✅ Features created: {len(feature_cols)} features")
        print(f"📊 Dataset shape: {self.features_df.shape}")
        return self.features_df

    def train_models(self):
        print("🤖 Training models...")

        if self.features_df is None:
            print("❌ No features available. Please create features first.")
            return None
        feature_cols = [col for col in self.features_df.columns
                       if col not in ['Close', 'Date']]
        X = self.features_df[feature_cols].values
        y = self.features_df['Close'].values

        if len(X) < 100:
            print("⚠️ Warning: Limited data for training. Results may not be reliable.")

        # Train-test split (80-20)
        split_idx = int(len(X) * 0.8)
        X_train, X_test = X[:split_idx], X[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]

        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        models_to_train = {
            'Linear Regression': LinearRegression(),
            'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
            'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
        }

        # Train models
        for name, model in models_to_train.items():
            print(f"  Training {name}...")
            try:
                if name == 'Linear Regression':
                    model.fit(X_train_scaled, y_train)
                    predictions = model.predict(X_test_scaled)
                else:
                    model.fit(X_train, y_train)
                    predictions = model.predict(X_test)

                # Calculate metrics
                rmse = np.sqrt(mean_squared_error(y_test, predictions))
                mae = mean_absolute_error(y_test, predictions)
                r2 = r2_score(y_test, predictions)

                # Store results
                self.models[name] = model
                self.results[name] = {
                    'RMSE': rmse,
                    'MAE': mae,
                    'R2': r2,
                    'predictions': predictions,
                    'actual': y_test
                }
                print(f"  ✅ {name} - RMSE: ${rmse:.2f}, R²: {r2:.3f}")
            except Exception as e:
                print(f"  ❌ Error training {name}: {e}")

        # Store test data
        self.test_data = {
            'X_test': X_test,
            'X_test_scaled': X_test_scaled,
            'y_test': y_test,
            'scaler': scaler,
            'feature_cols': feature_cols
        }
        return self.results

    def create_visualizations(self):
        print("📈 Creating visualizations...")

        if self.data is None:
            print("❌ No data available for visualization")
            return

        try:
            # 1. Price History
            fig1 = go.Figure()
            fig1.add_trace(go.Scatter(
                x=self.data['Date'],
                y=self.data['Close'],
                mode='lines',
                name='BTC Price',
                line=dict(color='orange', width=2)
            ))
            fig1.update_layout(
                title='Bitcoin Price History',
                xaxis_title='Date',
                yaxis_title='Price (USD)',
                template='plotly_white',
                height=400
            )
            fig1.show()

            # 2. Technical Indicators
            if self.features_df is not None and len(self.features_df) > 100:
                recent_data = self.features_df.tail(100)
                fig2 = make_subplots(
                    rows=2, cols=1,
                    subplot_titles=['Price with Moving Averages', 'RSI'],
                    vertical_spacing=0.1
                )

                # Price and MA
                fig2.add_trace(go.Scatter(x=recent_data['Date'], y=recent_data['Close'],
                                        name='Close', line=dict(color='blue')), row=1, col=1)
                if 'SMA_20' in recent_data.columns:
                    fig2.add_trace(go.Scatter(x=recent_data['Date'], y=recent_data['SMA_20'],
                                            name='SMA 20', line=dict(color='red', dash='dash')), row=1, col=1)

                # RSI
                if 'RSI' in recent_data.columns:
                    fig2.add_trace(go.Scatter(x=recent_data['Date'], y=recent_data['RSI'],
                                            name='RSI', line=dict(color='purple')), row=2, col=1)
                    fig2.add_hline(y=70, line_dash="dot", line_color="red", row=2, col=1)
                    fig2.add_hline(y=30, line_dash="dot", line_color="green", row=2, col=1)

                fig2.update_layout(height=600, title_text="Technical Analysis")
                fig2.show()

            # 3. Model Performance
            if self.results:
                model_names = list(self.results.keys())
                rmse_values = [self.results[model]['RMSE'] for model in model_names]
                r2_values = [self.results[model]['R2'] for model in model_names]

                fig3 = make_subplots(
                    rows=1, cols=2,
                    subplot_titles=['RMSE (Lower is Better)', 'R² Score (Higher is Better)']
                )

                fig3.add_trace(go.Bar(x=model_names, y=rmse_values, name='RMSE',
                                    marker_color='lightcoral'), row=1, col=1)
                fig3.add_trace(go.Bar(x=model_names, y=r2_values, name='R²',
                                    marker_color='lightblue'), row=1, col=2)

                fig3.update_layout(height=400, title_text="Model Performance Comparison")
                fig3.show()

                # 4. Predictions vs Actual
                best_model = min(self.results.keys(), key=lambda x: self.results[x]['RMSE'])
                predictions = self.results[best_model]['predictions']
                actual = self.results[best_model]['actual']

                fig4 = go.Figure()
                fig4.add_trace(go.Scatter(y=actual, mode='lines', name='Actual', line=dict(color='blue')))
                fig4.add_trace(go.Scatter(y=predictions, mode='lines', name=f'{best_model} Predictions',
                                        line=dict(color='red', dash='dash')))
                fig4.update_layout(
                    title=f'Predictions vs Actual - {best_model}',
                    xaxis_title='Time Steps',
                    yaxis_title='Price (USD)',
                    template='plotly_white',
                    height=400
                )
                fig4.show()
        except Exception as e:
            print(f"⚠️ Warning: Error creating some visualizations: {e}")

    def predict_next_price(self):
        print("🔮 Predicting next price...")

        if not self.results:
            print("❌ No trained models available")
            return None

        try:
            # Get best model
            best_model_name = min(self.results.keys(), key=lambda x: self.results[x]['RMSE'])
            best_model = self.models[best_model_name]

            # Get latest features
            latest_features = self.features_df[self.test_data['feature_cols']].iloc[-1:].values

            # Scale if needed
            if best_model_name == 'Linear Regression':
                latest_features = self.test_data['scaler'].transform(latest_features)

            # Predict
            prediction = float(best_model.predict(latest_features)[0])
            current_price = float(self.data['Close'].iloc[-1])
            change_pct = ((prediction - current_price) / current_price) * 100

            print(f"🎯 Best Model: {best_model_name}")
            print(f"💰 Current Price: ${current_price:.2f}")
            print(f"📈 Predicted Next Price: ${prediction:.2f}")
            print(f"📊 Expected Change: {change_pct:+.2f}%")

            return {
                'model': best_model_name,
                'current_price': current_price,
                'predicted_price': prediction,
                'change_percent': change_pct
            }
        except Exception as e:
            print(f"❌ Error making prediction: {e}")
            return None

    def generate_trading_signal(self):
        print("🎯 Generating trading signals...")

        if self.features_df is None:
            print("❌ No features available for signal generation")
            return ["HOLD"]

        try:
            latest_data = self.features_df.iloc[-1]
            signals = []

            # RSI signal
            if 'RSI' in latest_data and not pd.isna(latest_data['RSI']):
                rsi = latest_data['RSI']
                if rsi < 30:
                    signals.append("BUY (Oversold)")
                elif rsi > 70:
                    signals.append("SELL (Overbought)")

            # Moving average signal
            if 'Close_SMA_20_ratio' in latest_data and not pd.isna(latest_data['Close_SMA_20_ratio']):
                ma_ratio = latest_data['Close_SMA_20_ratio']
                if ma_ratio > 1.02:
                    signals.append("BUY (Above MA)")
                elif ma_ratio < 0.98:
                    signals.append("SELL (Below MA)")

            if not signals:
                signals.append("HOLD")

            print(f"📊 Trading Signals: {', '.join(signals)}")
            return signals
        except Exception as e:
            print(f"❌ Error generating signals: {e}")
            return ["HOLD"]

    def risk_analysis(self):
        print("⚠️ Performing risk analysis...")

        if self.data is None:
            print("❌ No data available for risk analysis")
            return None

        try:
            returns = self.data['Close'].pct_change().dropna()

            # Value at Risk (VaR)
            var_95 = np.percentile(returns, 5) * 100
            var_99 = np.percentile(returns, 1) * 100

            # Volatility metrics
            current_vol = returns.std() * np.sqrt(252) * 100  # Annualized

            # Maximum drawdown
            cumulative_returns = (1 + returns).cumprod()
            peak = cumulative_returns.expanding().max()
            drawdown = (cumulative_returns - peak) / peak
            max_drawdown = drawdown.min() * 100

            risk_metrics = {
                'VaR_95': var_95,
                'VaR_99': var_99,
                'Annual_Volatility': current_vol,
                'Max_Drawdown': max_drawdown
            }

            print(f"📊 95% VaR (daily): {var_95:.2f}%")
            print(f"📊 99% VaR (daily): {var_99:.2f}%")
            print(f"📊 Annual Volatility: {current_vol:.2f}%")
            print(f"📊 Max Drawdown: {max_drawdown:.2f}%")

            return risk_metrics
        except Exception as e:
            print(f"❌ Error in risk analysis: {e}")
            return None

    def run_complete_analysis(self):
        print("🚀 Starting Complete Bitcoin Analysis")
        print("=" * 50)

        try:
            # Step 1: Collect data
            if self.collect_data() is None:
                return None

            # Step 2: Create features
            if self.create_features() is None:
                return None

            # Step 3: Train models
            if self.train_models() is None:
                return None

            # Step 4: Create visualizations
            self.create_visualizations()

            # Step 5: Risk analysis
            risk_metrics = self.risk_analysis()

            # Step 6: Make predictions
            prediction = self.predict_next_price()

            # Step 7: Generate trading signals
            signals = self.generate_trading_signal()

            print("\n" + "=" * 50)
            print("✅ Complete Analysis Finished!")

            return {
                'prediction': prediction,
                'signals': signals,
                'risk_metrics': risk_metrics,
                'model_results': self.results
            }
        except Exception as e:
            print(f"❌ Error during analysis: {e}")
            return None

def compare_cryptos(symbols=['BTC-USD', 'ETH-USD', 'SOL-USD'], period='1y'):
    print(f"📊 Comparing cryptocurrencies: {', '.join([s.replace('-USD', '') for s in symbols])}")

    crypto_data = {}
    for symbol in symbols:
        try:
            data = yf.download(symbol, period=period, progress=False)
            if not data.empty:
                crypto_data[symbol.replace('-USD', '')] = data['Close']
        except Exception as e:
            print(f"❌ Error downloading {symbol}: {e}")

    if not crypto_data:
        print("❌ No data collected")
        return None, None

    try:
        # Create comparison DataFrame
        comparison_df = pd.DataFrame(crypto_data)

        # Calculate normalized performance (starting from 100)
        normalized_df = comparison_df / comparison_df.iloc[0] * 100

        # Calculate statistics
        returns_df = comparison_df.pct_change().dropna()
        stats = pd.DataFrame({
            'Total_Return_%': ((comparison_df.iloc[-1] / comparison_df.iloc[0] - 1) * 100).round(2),
            'Volatility_%': (returns_df.std() * np.sqrt(252) * 100).round(2),
            'Sharpe_Ratio': (returns_df.mean() / returns_df.std() * np.sqrt(252)).round(3)
        })

        print("\n📈 Cryptocurrency Comparison:")
        print(stats)

        # Plot comparison
        fig = go.Figure()
        for crypto in normalized_df.columns:
            fig.add_trace(go.Scatter(
                x=normalized_df.index,
                y=normalized_df[crypto],
                mode='lines',
                name=crypto,
                line=dict(width=2)
            ))

        fig.update_layout(
            title='Cryptocurrency Performance Comparison (Normalized to 100)',
            xaxis_title='Date',
            yaxis_title='Normalized Price',
            template='plotly_white',
            height=500
        )
        fig.show()

        return comparison_df, stats
    except Exception as e:
        print(f"❌ Error in crypto comparison: {e}")
        return None, None

# Main execution functions
def quick_analysis():
    """Quick Bitcoin analysis"""
    print("🚀 Running Quick Bitcoin Analysis...")
    predictor = CompleteBitcoinPredictor()
    results = predictor.run_complete_analysis()
    return predictor, results

def complete_analysis():
    print("🚀 Running Complete Bitcoin Analysis...")

    # Main analysis
    predictor, results = quick_analysis()

    if results is not None:
        # Additional crypto comparison
        print("\n🌍 Comparing with other cryptocurrencies...")
        try:
            compare_cryptos()
        except Exception as e:
            print(f"❌ Error in crypto comparison: {e}")

        # Generate summary
        print("\n📊 ANALYSIS SUMMARY")
        print("=" * 40)

        if results['prediction']:
            pred = results['prediction']
            print(f"💰 Current BTC Price: ${float(pred['current_price']):,.2f}")
            print(f"📈 Predicted Price: ${float(pred['predicted_price']):,.2f}")
            print(f"📊 Expected Change: {float(pred['change_percent']):+.2f}%")

        if results['signals']:
            print(f"🎯 Trading Signals: {', '.join(results['signals'])}")

        if results['risk_metrics']:
            risk = results['risk_metrics']
            print(f"⚠️ 95% VaR: {risk['VaR_95']:.2f}%")
            print(f"📊 Annual Volatility: {risk['Annual_Volatility']:.1f}%")

    return predictor, results

# Real-time update function
def get_real_time_update():
    print("🔄 Getting real-time Bitcoin update...")
    try:
        # Get current price
        btc_current = yf.download('BTC-USD', period='1d', interval='1m', progress=False)
        if not btc_current.empty:
            current_price = float(btc_current['Close'].iloc[-1])
            if len(btc_current) >= 60:
                price_1h_ago = float(btc_current['Close'].iloc[-60])
                change_1h = ((current_price - price_1h_ago) / price_1h_ago) * 100
            else:
                # If less than 60 minutes of data, use available data
                price_start = float(btc_current['Close'].iloc[0])
                change_1h = ((current_price - price_start) / price_start) * 100

            print(f"💰 Current BTC Price: ${current_price:.2f}")
            print(f"📊 1-Hour Change: {change_1h:+.2f}%")
            return current_price, change_1h
        else:
            print("❌ Could not fetch real-time data")
            return None, None
    except Exception as e:
        print(f"❌ Error getting real-time data: {e}")
        # Fallback to daily data
        try:
            print("🔄 Trying fallback method...")
            btc_daily = yf.download('BTC-USD', period='2d', progress=False)
            if not btc_daily.empty:
                current_price = float(btc_daily['Close'].iloc[-1])
                prev_price = float(btc_daily['Close'].iloc[-2])
                change_24h = ((current_price - prev_price) / prev_price) * 100
                print(f"💰 Current BTC Price: ${current_price:.2f}")
                print(f"📊 24-Hour Change: {change_24h:+.2f}%")
                return current_price, change_24h
            else:
                print("❌ Fallback method also failed")
                return None, None
        except Exception as e2:
            print(f"❌ Fallback error: {e2}")
            return None, None

# Display instructions and run analysis
print("🎉 Bitcoin Price Prediction System Ready!")
print("\n📋 Available Functions:")
print("1. quick_analysis() - Basic analysis with predictions")
print("2. complete_analysis() - Full comprehensive analysis")
print("3. compare_cryptos() - Compare different cryptocurrencies")
print("4. get_real_time_update() - Get current Bitcoin price and short-term change")
print("\n🚀 Starting Analysis Now...")
print("=" * 60)

# Run the complete analysis automatically
try:
    predictor, results = complete_analysis()
    print("\n🎯 Analysis completed successfully!")
    print("💡 The 'predictor' object contains all models and methods for further analysis.")

    # Get real-time update
    print("\n" + "=" * 40)
    get_real_time_update()

except Exception as e:
    print(f"❌ Error during analysis: {e}")
    print("💡 Try running quick_analysis() instead")

🎉 Bitcoin Price Prediction System Ready!

📋 Available Functions:
1. quick_analysis() - Basic analysis with predictions
2. complete_analysis() - Full comprehensive analysis
3. compare_cryptos() - Compare different cryptocurrencies
4. get_real_time_update() - Get current Bitcoin price and short-term change

🚀 Starting Analysis Now...
🚀 Running Complete Bitcoin Analysis...
🚀 Running Quick Bitcoin Analysis...
🚀 Starting Complete Bitcoin Analysis
📥 Collecting Bitcoin data...
✅ Data collected: 732 records
📅 Date range: 2023-05-26 to 2025-05-26
💰 Current price: $109515.35
🔧 Creating features...
❌ Error during analysis: Cannot set a DataFrame with multiple columns to the single column Close_SMA_5_ratio

🎯 Analysis completed successfully!
💡 The 'predictor' object contains all models and methods for further analysis.

🔄 Getting real-time Bitcoin update...
💰 Current BTC Price: $109462.70
📊 1-Hour Change: -0.69%
